In [ ]:
import matplotlib

from matplotlib import rc

rc("text", usetex=True)
rc("font", family="serif")

In [ ]:
import numpy
import scipy.optimize
import sympy

from matplotlib import pyplot

In [ ]:
def best_unif_app_dis_coef(func, ctrl):
    n = ctrl.size - 1
    
    lst = [ctrl**i for i in range(n)] + [(-1)**numpy.arange(n+1)]
    a = numpy.vstack(lst).transpose()
    
    b = func(ctrl)
    
    x = numpy.linalg.solve(a, b)
    
    c, d = x[:n], x[n]
    
    return c, d

In [ ]:
def poly_comb(coefficient):
    def poly(x):
        return numpy.sum(c * x**i for i, c in enumerate(coefficient))
    
    return poly

In [ ]:
def plot_func(func, left=-1.0, right=1.0, num=1000, ax=pyplot, **kwargs):
    x = numpy.linspace(left, right, num)
    y = func(x)
    ax.plot(x, y, **kwargs)

In [ ]:
def func(x):
    return numpy.abs(x**3)

In [ ]:
fig = pyplot.figure(figsize=(6., 4.))
ax = fig.add_subplot(1, 1, 1)

ct = numpy.cos(numpy.linspace(0.0, numpy.pi, 7))[::-1]

n = len(ct) - 1

plot_func(func, ax=ax, label="$f$")

for i in range(5):
    c, d = best_unif_app_dis_coef(func, ct)
    
    p = poly_comb(c)
    
    plot_func(p, ax=ax, label="$p_{}$".format(i+1))

    xi = [-1.0] + [scipy.optimize.brentq(
        lambda x: p(x) - func(x),
        ct[i], ct[i+1]
    ) for i in range(n)] + [1.0]

    sig = (-1) ** numpy.arange(n+1)
    if numpy.signbit(d):
        sig *= -1
    
    opt = [scipy.optimize.minimize(
        lambda x: sig[i] * (p(x) - func(x)),
        (xi[i] + xi[i+1]) / 2.0,
        method="L-BFGS-B",
        bounds=((xi[i], xi[i+1]),)
    ) for i in range(n+1)]
    
    err_list = [float(o["fun"]) for o in opt]
    err = max(err_list) - min(err_list)
    print("Step {0}, error {1}".format(i, err))
    if err < 1.0e-4:
        break
    
    ct = numpy.hstack([o["x"] for o in opt])

ax.set_xlim(-1.1, 1.1)
ax.set_ylim(-0.1, 1.1)

pyplot.legend()

pyplot.savefig("Figure01.pdf")
pyplot.savefig("Figure01.pgf")
pyplot.show()
pyplot.close()

In [ ]:
fig = pyplot.figure(figsize=(6., 4.))
ax = fig.add_subplot(1, 1, 1)

plot_func(lambda x: p(x) - func(x), ax=ax, label="$ p_{} - f $".format(i+1))

ax.set_xlim(-1.1, 1.1)
ax.set_ylim(-0.015, 0.015)

pyplot.legend()

pyplot.savefig("Figure02.pdf")
pyplot.savefig("Figure02.pgf")
pyplot.show()
pyplot.close()

In [ ]:
print(", ".join([str(p) for p in ct]))

In [ ]:
sympy.init_printing()

x = sympy.Symbol("x")

poly = sum([x**deg * coe for deg, coe in enumerate(c)])

print(sympy.latex(poly))